In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))


2025-11-14 16:34:58.588223: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-14 16:34:58.624811: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-14 16:35:00.166999: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
tf.test.is_gpu_available()


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


I0000 00:00:1763134500.973553    5395 gpu_device.cc:2020] Created device /device:GPU:0 with 9509 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4080 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


True

## Parte 1 - Construir el modelo de CNN

In [4]:
# Importar las librerías y paquetes
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential #Para inicializar los parámetros de la Red Neuronal
from tensorflow.keras.layers import Conv2D # Para crear una capa de convolución 2D
from tensorflow.keras.layers import MaxPooling2D # para el pooling
from tensorflow.keras.layers import Flatten # Para el aplanado
from tensorflow.keras.layers import Dense # Para declarar y crear las capas intermedias de la Red Neuronal
from tensorflow.keras.layers import Dropout # Para crear la capa de dropout (de olvido) - útil cuando tenemos mucha varianza - Evita overfitting desactivando algunas neurones de la capa inmediatamente anterior 


In [5]:
# Inicializar la CNN

classifier = Sequential() # Con esto se inicializa. No necesita ningún hiperparámetro

In [6]:
# Paso 1 - Convolución

classifier.add(Input(shape = (64, 64, 3)))
classifier.add(Conv2D(32, kernel_size = (3, 3), activation = "relu")) # la activación tiene que estar dentro de Conv2D
# forma de las imágenes ->input_shape =(64 filas, 64 columnas, 3 canales de color) 
# 32 filtros de 3 x 3


I0000 00:00:1763134500.996094    5395 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9509 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4080 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [7]:
# Paso 2 - Max Pooling
classifier.add(MaxPooling2D(pool_size =(2,2)))

In [8]:
# Para mejorar resultado, añado una segunda capa de convolución. No hace falta Input
# Tb añado otro Max pooling
classifier.add(Conv2D(32, kernel_size = (3, 3), activation = "relu"))
classifier.add(MaxPooling2D(pool_size =(2,2)))

In [9]:
# Paso 3 - Flattening / Aplanado

# La capa de entrada de la CNN tiene que ser un vector unidimensionalabs

classifier.add(Flatten())

In [10]:
# Paso 4 - Full Connection

classifier.add(Dense(units = 128, activation ="relu"))
classifier.add(Dense(units = 1, activation ="sigmoid")) # 1 porque es la probabolidad de ser perro. La del gato sería 1-p

In [11]:
# Compilar la CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=["accuracy"])

## Parte 2 - Ajustar la CNN a las imágenes para entrenar

In [12]:
# usaremos Keras para importar las imágenes

from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [13]:
# Preprocesamiento de las imágenes de entrenamiento
train_datagen = ImageDataGenerator(rescale = 1./255,  # Normalizando las imágenes en datos decimales de 0 a 1
                                   shear_range = 0.2,  # Aplicando transformación de corte. Sirve para que el modelo sea más robusto a pequeñas distorsiones
                                   zoom_range = 0.2,  # Aplicando aumento por zoom
                                   horizontal_flip = True)  # Aplicando volteo horizontal aleatorio

#Todo esto es data augmentation: crear variaciones de la misma imagen para que la red generalice mejor.


In [14]:
# Preprocesamiento de las imágenes de prueba
test_datagen = ImageDataGenerator(rescale = 1./255)  # Normalizando las imágenes de prueba

# Aquí solo reescalas (no haces augmentación), porque quieres medir el rendimiento sobre imágenes “reales”, no distorsionadas artificialmente.

In [15]:
# Cargando el conjunto de datos de entrenamiento
training_set = train_datagen.flow_from_directory('dataset/training_set',  # Ruta del conjunto de entrenamiento
                                                 target_size = (64, 64),  # Tamaño de las imágenes originales. redimensiona todas las imágenes a 64×64 píxeles.
                                                 batch_size = 32,  # Tamaño de lote: de 32 en 32 cada “paso de entrenamiento” usa 32 imágenes.
                                                 class_mode = 'binary')  # Clasificación binaria (dos clases: perros y gatos). Etiquetas serán 0/1

Found 8048 images belonging to 2 classes.


In [16]:
# Cargando el conjunto de datos de prueba
test_set = test_datagen.flow_from_directory('dataset/test_set',  # Ruta del conjunto de prueba
                                            target_size = (64, 64),  # Tamaño de las imágenes
                                            batch_size = 32,  # Tamaño de lote
                                            class_mode = 'binary')  # Clasificación binaria

Found 2001 images belonging to 2 classes.


In [17]:
# Ajustando la CNN al conjunto de entrenamiento
classifier.fit(training_set,  # Entrenando la CNN con el conjunto de entrenamiento
                         epochs = 25,  # Número de épocas
                         validation_data = test_set) # Conjunto de validación


Epoch 1/25


2025-11-14 16:35:04.822781: I external/local_xla/xla/service/service.cc:163] XLA service 0x7dcb4c016e20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-11-14 16:35:04.822806: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 4080 Laptop GPU, Compute Capability 8.9
2025-11-14 16:35:04.863285: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-11-14 16:35:05.040193: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91501
2025-11-14 16:35:05.073854: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-14 16:35:05.

  1/252 ━━━━━━━━━━━━━━━━━━━━ 15:50 4s/step - accuracy: 0.4375 - loss: 0.6997

I0000 00:00:1763134507.230941    5504 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


252/252 ━━━━━━━━━━━━━━━━━━━━ 185s 721ms/step - accuracy: 0.5779 - loss: 0.6716 - val_accuracy: 0.6327 - val_loss: 0.6421
Epoch 2/25
252/252 ━━━━━━━━━━━━━━━━━━━━ 50s 200ms/step - accuracy: 0.6499 - loss: 0.6286 - val_accuracy: 0.6852 - val_loss: 0.6110
Epoch 3/25
252/252 ━━━━━━━━━━━━━━━━━━━━ 51s 201ms/step - accuracy: 0.6876 - loss: 0.5853 - val_accuracy: 0.7156 - val_loss: 0.5592
Epoch 4/25
252/252 ━━━━━━━━━━━━━━━━━━━━ 54s 213ms/step - accuracy: 0.7116 - loss: 0.5571 - val_accuracy: 0.6992 - val_loss: 0.6025
Epoch 5/25
252/252 ━━━━━━━━━━━━━━━━━━━━ 54s 214ms/step - accuracy: 0.7353 - loss: 0.5275 - val_accuracy: 0.7436 - val_loss: 0.5231
Epoch 6/25
252/252 ━━━━━━━━━━━━━━━━━━━━ 54s 215ms/step - accuracy: 0.7546 - loss: 0.5061 - val_accuracy: 0.7521 - val_loss: 0.5044
Epoch 7/25
252/252 ━━━━━━━━━━━━━━━━━━━━ 53s 209ms/step - accuracy: 0.7691 - loss: 0.4841 - val_accuracy: 0.7451 - val_loss: 0.5334
Epoch 8/25
252/252 ━━━━━━━━━━━━━━━━━━━━ 54s 215ms/step - accuracy: 0.7709 - loss: 0.4790 - va

KeyboardInterrupt: 

#Versión moderna
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

IMG_SIZE = (64, 64)
BATCH_SIZE = 32

# Cargar datasets desde directorios (tf.data)
train_ds = keras.utils.image_dataset_from_directory(
    "dataset/training_set",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="binary",
    shuffle=True,
    seed=42,
)

val_ds = keras.utils.image_dataset_from_directory(
    "dataset/test_set",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="binary",
    shuffle=False,
)

# Mejora de rendimiento (pipeline)
train_ds = train_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

# Data augmentation como capas del modelo
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ],
    name="data_augmentation",
)

# Ejemplo de modelo CNN moderno (puedes sustituir por tu 'classifier')
inputs = keras.Input(shape=(*IMG_SIZE, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1.0 / 255)(x)
x = layers.Conv2D(32, (3, 3), activation="relu")(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

classifier = keras.Model(inputs, outputs)

classifier.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

history = classifier.fit(
    train_ds,
    epochs=25,
    validation_data=val_ds,
)






# Ejercicio

## 1) Predicción de una sola imagen

In [ ]:
import numpy as np
import os
from tensorflow.keras.utils import load_img, img_to_array

# Ruta de una imagen concreta
img_path = "dataset/single_prediction/cendre.jpg"  

# 1. Cargar la imagen con el mismo tamaño que entrenaste
img = load_img(img_path, target_size=(64, 64))# la convierto a 64x64

# 2. Convertir a array
img_array = img_to_array(img)

# 3. Reescalar igual que en ImageDataGenerator (rescale=1./255)
img_array = img_array / 255.0

# 4. Añadir dimensión batch: (64,64,3) -> (1,64,64,3)
img_array = np.expand_dims(img_array, axis=0) #Porque keras no acepta imágenes sueltas sino batches

# 5. Predecir
prob = classifier.predict(img_array).item()  #convierte un array en número

if prob > 0.5:
    print("dog")
else:
    print("cat")




# Solución profesor

In [ ]:
import numpy as np #para procesar la imagen
from tensorflow.keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg',target_size =(64,64))

In [ ]:
#Como el input_shape = (64,64,3)
test_image = image.img_to_array(test_image)

In [ ]:
#Hay que añadirle otra dimensión correspondiente al lote
test_image = np.expand_dims(test_image, axis=0) # expande la 4a dimensión en el primer eje -> (1,64,64,3)

In [ ]:
result= classifier.predict(test_image)
training_set.class_indices # así se que 'cats': 0 y 'dogs': 1
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'